In [ ]:
# Преобразование pdf в каритнку png
import fitz  # PyMuPDF
from PIL import Image

def convert_pdf_to_high_resolution_images(pdf_path, scale_factor=2):
    pdf_document = fitz.open(pdf_path)

    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]
        
        pix = page.get_pixmap(matrix=fitz.Matrix(scale_factor, scale_factor))
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        output_path = f'image_{page_number + 1}.png'
        img.save(output_path, 'PNG')

pdf_path = 'test_pas.pdf'
convert_pdf_to_high_resolution_images(pdf_path, scale_factor=2)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def extract_rectangular_regions(image_path, output_path='rectangular_regions.jpg'):
    image = cv2.imread(image_path)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Применение адаптивного порогового преобразования для выделения контуров
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 15)
    
    # Поиск контуров на изображении
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Создание маски
    mask = np.zeros_like(gray)
    
    # Проход по всем контурам и нарисовать их на маске
    for contour in contours:
        cv2.drawContours(mask, [contour], 0, (255), thickness=cv2.FILLED)
    
    # Применение маски к изображению
    result = cv2.bitwise_and(image, image, mask=mask)
    
    # Сохранение изображения с выделенными прямоугольными областями
    cv2.imwrite(output_path, result)

    # Отображение изображения только с прямоугольными областями
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title('Image with Only Rectangular Regions')
    plt.show()

# Пример использования
image_path = 'image_2.png'
output_path = 'rectangular_regions.jpg'
extract_rectangular_regions(image_path, output_path)
print(f"Прямоугольные области сохранены по пути: {output_path}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Загрузка и подготовка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Нормализация значений пикселей к диапазону [0, 1]
train_images, valid_images, test_images = train_images / 255.0, valid_images / 255.0, test_images / 255.0

# Создание сети
model = models.Sequential()
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model.fit(train_images, train_labels, epochs=10, validation_data=(valid_images, valid_labels))

# Сохранение обученной модели
model.save('mnist_digit_recognition_model.h5')

# Оценка модели
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

# Визуализация результатов
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Пример распознавания
predictions = model.predict(test_images[:5])
predicted_labels = [tf.argmax(prediction) for prediction in predictions]

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(test_images[i], cmap='gray')
    plt.title(f"Predicted: {predicted_labels[i]}, Actual: {test_labels[i]}")
    plt.axis('off')

plt.show()


In [ ]:
import cv2
import pytesseract
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Установка пути к исполняемому файлу Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)

    binary = binary / 255.0

    return binary

def create_model(input_shape, num_classes):
    model = Sequential()

    # Входной слой LSTM
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))

    # Полносвязный слой для каждого временного шага
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

    # Объединение результатов
    model.add(Flatten())

    # Общий выходной слой
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def train_model(model, X_train, y_train, epochs=10):
    model.fit(X_train, y_train, epochs=epochs, batch_size=64)

image_path = 'buf.png'
text = pytesseract.image_to_string(Image.open(image_path), lang='rus')
print("Исходный текст изображения:")
print(text)

binary_image = preprocess_image(image_path)

input_shape = (binary_image.shape[0], binary_image.shape[1], 1)  # Формат входных данных для LSTM
num_classes = 10

model = create_model(input_shape, num_classes)

binary_image_reshaped = np.expand_dims(binary_image, axis=0)

# Обучение модели
train_model(model, binary_image_reshaped, y_train, epochs=10)
